# Processing California's public payrolls

This notebook processess annual government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [1]:
import pandas as pd
import zipfile
from urllib.request import urlopen 
import pyarrow
import cpi
import os
import glob
import io
import requests
import matplotlib
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Download zipped salary tables by year and agency type

In [2]:
# os.chdir('/Users/mhustiles/data/data/controller/input/')

In [3]:
# formaturl = lambda x: 'https://publicpay.ca.gov/RawExport/' + f'{x[1]}_' + f'{x[0]}' + '.zip'

In [4]:
# metadata = []
# for y in range(2009,2019):
#     for e in ['City', 'County', 'SpecialDistrict', 'StateDepartment']:
# #     for e in ['City', 'County']:
#         metadata.append(dict(entity = e, year = y, url = formaturl((e, y))))

### Extract CSVs from .zip files, and then discard the .zip files

In [5]:
# for m in metadata:
#     !wget '{m['url']}'
#     !unzip \*.zip
#     !rm -f *.zip

---

### Read all the text files, loop and store them in a dataframe

In [6]:
path = '/Users/mhustiles/data/data/controller/input/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, encoding = "ISO-8859-1", \
            low_memory=False, dtype = {'DepartmentOrSubdivision': 'object', 'Year': 'object'}) for f in all_files)
concatenated_df = pd.concat(df_from_each_file, ignore_index=True)

### Trim the dataframe to the columns we need

In [17]:
payroll = \
pd.DataFrame(concatenated_df[['Year','EmployerType','EmployerPopulation','EmployerName','DepartmentOrSubdivision',\
                 'Position','OvertimePay','TotalWages', 'TotalRetirementAndHealthContribution', 'EmployerCounty']])

### Not everyone reports (or pays) overtime

In [18]:
payroll['OvertimePay'].fillna(0, inplace = True)
payroll['DepartmentOrSubdivision'].fillna('NOT LISTED', inplace = True)
payroll['EmployerPopulation'].fillna(0, inplace = True)

### Clean up column headers

In [19]:
payroll.columns = payroll.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [20]:
payroll.rename(columns = {
'employertype':'type',
'employerpopulation':'population',
'employername':'employer',
'departmentorsubdivision':'department',
'overtimepay':'overtime',
'totalretirementandhealthcontribution':'benefits',
'totalwages':'wages',
 }, inplace = True)

### Uppercase everything because their title casing across hundreds of agencies is janky

In [21]:
payroll = payroll.apply(lambda x: x.astype(str).str.upper())

### Convert year column to datetime year for CPI function

In [22]:
payroll['year'] = pd.to_datetime(payroll['year'])

In [23]:
payroll['year'] = payroll['year'].dt.year

### Adjust overtime on each row for CPI according to its respective year

In [25]:
payroll[['overtime', 'population', 'wages', 'benefits']] = \
payroll[['overtime', 'population', 'wages', 'benefits']].astype(float)

In [26]:
payroll['basewages'] = (payroll['wages'] - payroll['overtime']).astype(float)

In [27]:
payroll['adjusted_overtime'] = payroll.apply(lambda x: cpi.inflate(x.overtime, x.year), axis=1)

In [28]:
payroll['adjusted_wages'] = payroll.apply(lambda x: cpi.inflate(x.wages, x.year), axis=1)

In [29]:
payroll['adjusted_benefits'] = payroll.apply(lambda x: cpi.inflate(x.benefits, x.year), axis=1)

In [30]:
payroll['adjusted_basewages'] = payroll.apply(lambda x: cpi.inflate(x.basewages, x.year), axis=1)

### How does the dataframe look? 

In [31]:
payroll.head()

,year,type,population,employer,department,position,overtime,wages,benefits,employercounty,basewages,adjusted_overtime,adjusted_wages,adjusted_benefits,adjusted_basewages
0,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTANT-AUDITOR,0,"4,120","1,088",ALAMEDA,"4,120",0,"4,744","1,253","4,744"
1,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ACCOUNTING TECHNICIAN,0,"54,438","14,031",ALAMEDA,"54,438",0,"62,689","16,158","62,689"
2,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,ADMIN SPECIALIST II,0,"25,376","5,486",ALAMEDA,"25,376",0,"29,222","6,318","29,222"
3,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0,"31,225","8,710",ALAMEDA,"31,225",0,"35,958","10,030","35,958"
4,2010,COUNTY,"1,517,756",ALAMEDA,AC CENTRAL COLLECTIONS,AUDITOR ASSOCIATE I,0,"27,677","4,916",ALAMEDA,"27,677",0,"31,872","5,661","31,872"


### How many records do we have here?

In [32]:
# How many records?
len(payroll)

10826479

### Clean up some employer names

In [33]:
payroll['employer']\
    .replace('SACRAMENTO METROPOLITAN FIRE DISTRICT', 'SACRAMENTO METRO FIRE', inplace=True)

In [34]:
payroll['employer']\
    .replace('FORESTRY & FIRE PROTECTION, CALIFORNIA DEPARTMENT OF', 'CALFIRE', inplace=True)

In [35]:
payroll['type']\
    .replace('STATE DEPARTMENT', 'STATE', inplace=True)

In [36]:
payroll['employer']\
    .replace('ORANGE COUNTY FIRE AUTHORITY', 'OC FIRE AUTHORITY', inplace=True)

### Export table of all titles to help ID fire roles

In [37]:
positions_all = payroll.groupby(['position', 'employer', 'type', 'department'])\
    .agg('size').reset_index()

In [38]:
positions_all['position'] = positions_all['position']\
                            .str.replace('!ST ', '', regex=False)\
                            .str.replace('1ST ', '', regex=False)\
                            .str.replace('2ND', '', regex=False)\
                            .str.replace('.E.', '', regex=False)\
                            .str.replace(' - ', '', regex=False)\
                            .str.replace('\d', '')\
                            .str.replace('[.!?\\-]', '')

In [39]:
positions_fire = positions_all[ (positions_all['position'].str.contains('FIRE')) |\
                                       (positions_all['type'].str.contains('FIRE')) |\
                                       (positions_all['department'].str.contains('FIRE')) |\
                                       (positions_all['employer'].str.contains('FIRE'))]

In [40]:
positions_fire_grouped = positions_fire.groupby(['position']).agg('size')

In [41]:
positions_fire_grouped.to_csv('/Users/mhustiles/data/github/notebooks/payroll/input/positions_fire_grouped.csv', header=True)

---

### Export to a lightweight format

In [42]:
payroll.reset_index().to_feather('/Users/mhustiles/data/data/controller/output/payroll.feather')

In [43]:
payroll.reset_index().to_csv('/Users/mhustiles/data/data/controller/output/payroll.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx